In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from datetime import datetime
import nltk
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
eng_stopwords = nltk.corpus.stopwords.words('english')
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import decomposition
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("cleaned_and_merged.csv").drop('Unnamed: 0', axis = 1)
df.head()

,author,contents,description,publisher,source_url,title,date,time,label
0,Stripe.com,a complete payments platform engineered growth...,"At Stripe, we’ve long been excited about the p...",Stripe.com,https://stripe.com/blog/ending-bitcoin-support,Ending Bitcoin Support,2018-01-23,00:00:00,0.0
1,Editorial Team,as scrambles serve massively expanding userbas...,As it scrambles to serve a massively expanding...,Finextra.com,https://www.finextra.com/newsarticle/31558/coi...,Coinbase hires former Twitter exec to lead cus...,2018-01-23,00:01:00,0.0
2,Scott Scanlon,so many cryptocurrencies so much money made lo...,So many cryptocurrencies. So much money to be ...,Youbrandinc.com,https://www.youbrandinc.com/crytocurrency/shou...,Should you buy bitcoin? Or Ethereum? Or Dash? ...,2018-01-23,00:03:12,0.0
3,http://www.dailymail.co.uk/home/search.html?s=...,by press association published edt january upd...,The Tokyo-based firm has been awarded a paymen...,Daily Mail,http://www.dailymail.co.uk/wires/pa/article-53...,World´s biggest Bitcoin exchange wins backing ...,2018-01-23,00:05:47,0.0
4,Phil Glazer,initial coin offerings icos regulatory wild we...,Initial coin offerings (ICOs) are a regulatory...,Hackernoon.com,https://hackernoon.com/is-regulation-needed-fo...,Is Regulation Needed for Institutional Investo...,2018-01-23,00:06:02,0.0


In [28]:
df['words'] = df['contents'].str.split()
    
df.head()

,author,contents,description,publisher,source_url,title,date,time,label,words
0,Stripe.com,a complete payments platform engineered growth...,"At Stripe, we’ve long been excited about the p...",Stripe.com,https://stripe.com/blog/ending-bitcoin-support,Ending Bitcoin Support,2018-01-23,00:00:00,0.0,"[a, complete, payments, platform, engineered, ..."
1,Editorial Team,as scrambles serve massively expanding userbas...,As it scrambles to serve a massively expanding...,Finextra.com,https://www.finextra.com/newsarticle/31558/coi...,Coinbase hires former Twitter exec to lead cus...,2018-01-23,00:01:00,0.0,"[as, scrambles, serve, massively, expanding, u..."
2,Scott Scanlon,so many cryptocurrencies so much money made lo...,So many cryptocurrencies. So much money to be ...,Youbrandinc.com,https://www.youbrandinc.com/crytocurrency/shou...,Should you buy bitcoin? Or Ethereum? Or Dash? ...,2018-01-23,00:03:12,0.0,"[so, many, cryptocurrencies, so, much, money, ..."
3,http://www.dailymail.co.uk/home/search.html?s=...,by press association published edt january upd...,The Tokyo-based firm has been awarded a paymen...,Daily Mail,http://www.dailymail.co.uk/wires/pa/article-53...,World´s biggest Bitcoin exchange wins backing ...,2018-01-23,00:05:47,0.0,"[by, press, association, published, edt, janua..."
4,Phil Glazer,initial coin offerings icos regulatory wild we...,Initial coin offerings (ICOs) are a regulatory...,Hackernoon.com,https://hackernoon.com/is-regulation-needed-fo...,Is Regulation Needed for Institutional Investo...,2018-01-23,00:06:02,0.0,"[initial, coin, offerings, icos, regulatory, w..."


In [35]:
indices2 = []

for i in df.index:
    if type(df['contents'][i]) is not str:
        indices.append(i)

len(indices2)
#This was previously 69

0

In [34]:
df = df.drop(indices)
df = df.reset_index()
df = df.drop('index', axis = 1)
df.head()

,author,contents,description,publisher,source_url,title,date,time,label,words
0,Stripe.com,a complete payments platform engineered growth...,"At Stripe, we’ve long been excited about the p...",Stripe.com,https://stripe.com/blog/ending-bitcoin-support,Ending Bitcoin Support,2018-01-23,00:00:00,0.0,"[a, complete, payments, platform, engineered, ..."
1,Editorial Team,as scrambles serve massively expanding userbas...,As it scrambles to serve a massively expanding...,Finextra.com,https://www.finextra.com/newsarticle/31558/coi...,Coinbase hires former Twitter exec to lead cus...,2018-01-23,00:01:00,0.0,"[as, scrambles, serve, massively, expanding, u..."
2,Scott Scanlon,so many cryptocurrencies so much money made lo...,So many cryptocurrencies. So much money to be ...,Youbrandinc.com,https://www.youbrandinc.com/crytocurrency/shou...,Should you buy bitcoin? Or Ethereum? Or Dash? ...,2018-01-23,00:03:12,0.0,"[so, many, cryptocurrencies, so, much, money, ..."
3,http://www.dailymail.co.uk/home/search.html?s=...,by press association published edt january upd...,The Tokyo-based firm has been awarded a paymen...,Daily Mail,http://www.dailymail.co.uk/wires/pa/article-53...,World´s biggest Bitcoin exchange wins backing ...,2018-01-23,00:05:47,0.0,"[by, press, association, published, edt, janua..."
4,Phil Glazer,initial coin offerings icos regulatory wild we...,Initial coin offerings (ICOs) are a regulatory...,Hackernoon.com,https://hackernoon.com/is-regulation-needed-fo...,Is Regulation Needed for Institutional Investo...,2018-01-23,00:06:02,0.0,"[initial, coin, offerings, icos, regulatory, w..."


In [36]:
### Word2Vec ###
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 50   # ignore all words with total frequency lower than this 
num_workers = 4       # Number of threads to run in parallel
context = 10         # Context window size                                                                                    

# Initialize and train the model (this will take some time)
from gensim.models import word2vec


print("Training word2vec model... ")
model = word2vec.Word2Vec(df['words'], workers=num_workers, \
           size=num_features, min_count = min_word_count, \
            window = context)
print("Done training model")


# save the model for later use. You can load it later using Word2Vec.load()
model_name = "weds344pm"
model.save(model_name)

Training word2vec model... 
Done training model


In [41]:
vocab = list(model.wv.vocab)

In [71]:
x = model[vocab]

In [80]:
x.shape

(18096, 300)